In [2]:
import seaborn as sns 
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession 
from pyspark.ml.feature import RegexTokenizer 
from pyspark.ml.feature import HashingTF, IDF 
from pyspark.ml.feature import StopWordsRemover,StringIndexer
from pyspark.sql.functions import concat_ws, col
from pyspark.ml.classification import LogisticRegression 
from pyspark.ml.evaluation import MulticlassClassificationEvaluator 

PySpark'ı başlatmak için kullanılan bir SparkSession oluşturuyoruz.

In [2]:
spark = SparkSession.builder.master("local[*]")\
                            .appName("ITC")\
                            .getOrCreate()

spark

In [3]:
#spark.stop()

Veri setini okuyor ve bir DataFrame oluşturuyoruz.

In [4]:
df = spark.read.csv("DATA/karisik_veri.csv", inferSchema=True, header=True)

df.show()

+---------------+--------------------+--------------------+
|       Kategori|              Başlık|              İçerik|
+---------------+--------------------+--------------------+
|           Spor|Hakkarigücü Kadın...|Turkcell Kadın Fu...|
|Bilim Teknoloji|Uzay raporu: Mars...|Hakemli bilimsel ...|
|         Sağlık|Gaziantep Şehir H...|Gaziantep Şehir H...|
|         Sağlık|Türkiye sınırın d...|Fırat Kalkanı Har...|
|        Ekonomi|3,2 milyar liralı...|Aile ve Sosyal Hi...|
|           Spor|Gençler Kılıç Dün...|Eskrimde, Gençler...|
|         Sağlık|"Gıda mühendisind...|"Mutfakta yapılan...|
|           Spor|Michael Schumache...|Formula 1'in ünlü...|
|         Eğitim|"Türkiye ile Güne...|Milli Eğitim Baka...|
|     ÖZEL HABER|7 Güzel Adam'ın b...|7 Güzel Adam'ın b...|
|         Sağlık|DSÖ, COVID-19'un ...|Dünya Sağlık Örgü...|
|           Spor|Kupada son çeyrek...|Ziraat Türkiye Ku...|
|         Eğitim|Meslek liselerind...|Milli Eğitim Baka...|
|   Kültür-Sanat|Dünyaca ünlü İngi...|Dü

In [5]:
print("Toplam veri sayısı: ", df.count())

Toplam veri sayısı:  29861


In [6]:
print("Kategori sütunundaki eksik değerlerin sayısı: ", df.toPandas()['Kategori'].isnull().sum())
print("Başlık sütunundaki eksik değerlerin sayısı: ", df.toPandas()['Başlık'].isnull().sum())
print("İçerik sütunundaki eksik değerlerin sayısı: ", df.toPandas()['İçerik'].isnull().sum())

Kategori sütunundaki eksik değerlerin sayısı:  0
Başlık sütunundaki eksik değerlerin sayısı:  91
İçerik sütunundaki eksik değerlerin sayısı:  169


In [7]:
df = df.dropna(subset=('Başlık'))
df = df.dropna(subset=('İçerik'))

In [8]:
print("Toplam veri sayısı: ", df.count())

Toplam veri sayısı:  29692


DataFrame içindeki "Başlık" ve "İçerik" adlı sütunlardaki verileri birleştirerek yeni bir "Text" sütunu oluşturuyoruz.

In [9]:
df = df.withColumn("Text", concat_ws(" ", "Başlık", 'İçerik'))

df = df.select('Kategori', 'Text')
df.show(10)

+---------------+--------------------+
|       Kategori|                Text|
+---------------+--------------------+
|           Spor|Hakkarigücü Kadın...|
|Bilim Teknoloji|Uzay raporu: Mars...|
|         Sağlık|Gaziantep Şehir H...|
|         Sağlık|Türkiye sınırın d...|
|        Ekonomi|3,2 milyar liralı...|
|           Spor|Gençler Kılıç Dün...|
|         Sağlık|"Gıda mühendisind...|
|           Spor|Michael Schumache...|
|         Eğitim|"Türkiye ile Güne...|
|     ÖZEL HABER|7 Güzel Adam'ın b...|
+---------------+--------------------+
only showing top 10 rows



RegexTokenizer kullanarak metin verilerini belirli bir desene göre parçalara ayırarak kelimeleri liste halinde elde ediyoruz.

In [10]:
df = df.drop("words")

tokenizer = RegexTokenizer(inputCol="Text", outputCol="words",pattern="\\s+|[,.;!?\\\\-]+|(?<!\\\\d)-|-(?!\\\\d)|[^\\w\\d\\u0130\\u0131\\u00c7\\u00e7\\u011e\\u011f\\u015e\\u015f\\u00d6\\u00f6\\u00dc\\u00fc]+")

df = tokenizer.transform(df)

df.select(['Kategori','Text', 'words']).show(5)

+---------------+--------------------+--------------------+
|       Kategori|                Text|               words|
+---------------+--------------------+--------------------+
|           Spor|Hakkarigücü Kadın...|[hakkarigücü, kad...|
|Bilim Teknoloji|Uzay raporu: Mars...|[uzay, raporu, ma...|
|         Sağlık|Gaziantep Şehir H...|[gaziantep, şehir...|
|         Sağlık|Türkiye sınırın d...|[türkiye, sınırın...|
|        Ekonomi|3,2 milyar liralı...|[3, 2, milyar, li...|
+---------------+--------------------+--------------------+
only showing top 5 rows



Doğal dil işlemede sıklıkla kullanılan nltk kütüphanesinden Türkçe stop words'leri (durak kelimeleri) çekiyoruz.
Örneğin "acaba ama aslında ve birkaç biz şey gibi"

In [11]:
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
turkish_stopwords = stopwords.words("turkish")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ereny\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Metindeki durak kelimeleri çıkararak metnin daha anlamlı kısımlarına odaklanmayı sağlıyoruz.

In [12]:
df = df.drop("filtered_words")

stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words", stopWords=turkish_stopwords)
df = stopwords_remover.transform(df)
df .select([ 'Kategori' , 'Text' , 'words' , 'filtered_words' ]).show(5)

+---------------+--------------------+--------------------+--------------------+
|       Kategori|                Text|               words|      filtered_words|
+---------------+--------------------+--------------------+--------------------+
|           Spor|Hakkarigücü Kadın...|[hakkarigücü, kad...|[hakkarigücü, kad...|
|Bilim Teknoloji|Uzay raporu: Mars...|[uzay, raporu, ma...|[uzay, raporu, ma...|
|         Sağlık|Gaziantep Şehir H...|[gaziantep, şehir...|[gaziantep, şehir...|
|         Sağlık|Türkiye sınırın d...|[türkiye, sınırın...|[türkiye, sınırın...|
|        Ekonomi|3,2 milyar liralı...|[3, 2, milyar, li...|[3, 2, milyar, li...|
+---------------+--------------------+--------------------+--------------------+
only showing top 5 rows



Hashing Term Frequency dönüştürücüsünü kullanarak veri setindeki kelimelerin frekanslarını hesaplıyoruz.

In [13]:
df = df.drop("raw_features")
hashing_tf = HashingTF(inputCol="filtered_words",
                       outputCol="raw_features", 
                       numFeatures=10000) # Deneme yanılma ile bulunur..

featurized_data = hashing_tf.transform(df)

TF-IDF, bir belgedeki bir kelimenin önemini ölçen bir istatistiksel değerdir. Bu değer, bir kelimenin bir belgedeki sıklığını (TF) ve tüm belgelerdeki görünüm sıklığını (IDF) dikkate alarak hesaplanır. 


In [14]:
idf = IDF(inputCol="raw_features", outputCol="features")

idf_vectorizer = idf.fit(featurized_data)

rescaled_data = idf_vectorizer.transform(featurized_data)

rescaled_data.select("Kategori",'Text', 'words', 'filtered_words', "features").show() 

+---------------+--------------------+--------------------+--------------------+--------------------+
|       Kategori|                Text|               words|      filtered_words|            features|
+---------------+--------------------+--------------------+--------------------+--------------------+
|           Spor|Hakkarigücü Kadın...|[hakkarigücü, kad...|[hakkarigücü, kad...|(10000,[317,580,6...|
|Bilim Teknoloji|Uzay raporu: Mars...|[uzay, raporu, ma...|[uzay, raporu, ma...|(10000,[134,202,3...|
|         Sağlık|Gaziantep Şehir H...|[gaziantep, şehir...|[gaziantep, şehir...|(10000,[86,746,81...|
|         Sağlık|Türkiye sınırın d...|[türkiye, sınırın...|[türkiye, sınırın...|(10000,[527,679,9...|
|        Ekonomi|3,2 milyar liralı...|[3, 2, milyar, li...|[3, 2, milyar, li...|(10000,[474,503,1...|
|           Spor|Gençler Kılıç Dün...|[gençler, kılıç, ...|[gençler, kılıç, ...|(10000,[903,1318,...|
|         Sağlık|"Gıda mühendisind...|[gıda, mühendisin...|[gıda, mühendisin...|(1

Kategori sütununu sayısal tipe dönüştürüyoruz.
Spor -> 0 , Eğitim -> 1 , Ekonomi -> 2 , Sağlık -> 3 , Kültür-Sanat -> 4 , Bilim Teknoloji -> 5

In [15]:
indexer = StringIndexer(inputCol="Kategori", outputCol="label")

rescaled_data = indexer.fit(rescaled_data).transform(rescaled_data)

Veri setini eğitim ve test olmak üzere bölüyoruz.

In [16]:
(train, test) = rescaled_data.randomSplit([0.75, 0.25], seed = 202)
print("Eğitim Veri seti Sayısı: " + str(train.count()))
print("Test Veri seti Sayısı: " + str(test.count()))

Eğitim Veri seti Sayısı: 22227
Test Veri seti Sayısı: 7465


Çok sınıflı (multinomial) bir Lojistik Regresyon modeli oluşturuyor ve eğitiyoruz. Daha sonra, bu modeli kullanarak test veri seti üzerinde tahminler yaparak tahmin sonuçlarını içeren bir DataFrame oluşturuyoruz.

In [17]:
lr = LogisticRegression(featuresCol='features',
                        labelCol='label',
                        family="multinomial",
                        regParam=0.3,
                        elasticNetParam=0,
                        maxIter=50)

lrModel = lr.fit(train)

predictions = lrModel.transform(test)

predictions.select("Text", 'probability','prediction', 'Kategori').show()

+--------------------+--------------------+----------+---------------+
|                Text|         probability|prediction|       Kategori|
+--------------------+--------------------+----------+---------------+
|"""100 bin dolara...|[0.04153611058084...|       5.0|Bilim Teknoloji|
|"""Ateş Şövalyesi...|[0.06824226459827...|       5.0|Bilim Teknoloji|
|"""Bir Adam Yarat...|[0.00910351432343...|       4.0|Bilim Teknoloji|
|"""Sifon çekmek C...|[0.02571815846943...|       3.0|Bilim Teknoloji|
|"""TCG Batıray"" ...|[0.17680872832889...|       5.0|Bilim Teknoloji|
|"""Take Off İstan...|[0.11833692461738...|       5.0|Bilim Teknoloji|
|"""Tapu Güvenilir...|[0.04031760668706...|       2.0|Bilim Teknoloji|
|"""Türkiye siber ...|[0.09402042515622...|       5.0|Bilim Teknoloji|
|"Antarktika'da ""...|[0.02948844698268...|       5.0|Bilim Teknoloji|
|"Avrupa'nın ""bul...|[0.08606331739154...|       4.0|Bilim Teknoloji|
|"Belki şurada küç...|[0.06545644932865...|       5.0|Bilim Teknoloji|
|"Botl

In [18]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")

print("Test-seti doğruluğu : ", evaluator.evaluate(predictions))

Test-seti doğruluğu :  0.8694675560211187
